In [27]:
import torch
from LSTM import LSTM
from utils_LSTM import params
import pickle
import io
from traffic_data_loader import data_loader_full, tensor_reshape, Traffic_Flow_Data
import numpy as np

In [28]:
# retreive parameters from params
input_size = params['input_size']
hidden_size = params['hidden_size']
num_layers = params['num_layers']
output_size = params['output_size']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
# load model
model_LSTM = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
model_LSTM.load_state_dict(torch.load('saved_model/model_LSTM.pth', map_location=device))
model_LSTM.to(device)
model_LSTM.eval() # enable evaluation mode

LSTM(
  (lstm): LSTM(17, 100, num_layers=3, batch_first=True)
  (fc): Linear(in_features=100, out_features=17, bias=True)
)

In [30]:
# start prediction
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)


file_path = '../Model_Final/Predicted/results.pkl'
with open(file_path, 'rb') as pickle_file:
    pred = CPU_Unpickler(pickle_file).load()
data = pred['flow_recon'].to(device)

data_occupancy_all, data_flow_all, data_speed_all = data_loader_full()
X_occu_all, _ = data_occupancy_all[:, :2], data_occupancy_all[:, 2]
X_occu_all = torch.tensor(X_occu_all, dtype=torch.float32).to(device)

data = torch.cat((X_occu_all, data), dim=1).detach()

data = tensor_reshape(data)

data_validation = data[int(data.size(0) * 0.7):, :]

data_val = Traffic_Flow_Data(data_validation, window_size = params['window_size'])

## 3 minutes prediction

In [31]:
# create empty np array
Y_VAL_1 = []
Y_PRED_1 = []

In [32]:
for idx in range(data_val.__len__()):
    x_val, y_val = data_val.__getitem__(idx)
    x_val = x_val.unsqueeze(0).to(device)
    y_pred = model_LSTM(x_val).detach().to('cpu').numpy()
    y_val = y_val.detach().to('cpu').numpy()
    Y_VAL_1.append(y_val)
    Y_PRED_1.append(y_pred)

KeyboardInterrupt: 

In [9]:
Y_VAL_1 = np.vstack(Y_VAL_1)
Y_PRED_1 = np.vstack(Y_PRED_1)

In [10]:
# calculate the RMSE and MAPE
rmse_flow_1_LSTM = np.sqrt(np.nanmean((Y_VAL_1 - Y_PRED_1) ** 2))
print(rmse_flow_1_LSTM)

mape_flow_1_LSTM = np.nanmean(np.abs((Y_VAL_1 - Y_PRED_1) / Y_VAL_1)) * 100
print(mape_flow_1_LSTM)

8.632509
11.774547


## 6 minutes prediction

In [11]:
# create empty np array
Y_VAL_2 = []
Y_PRED_2 = []

In [12]:
for idx in range(data_val.__len__()-1):
    x_val, _ = data_val.__getitem__(idx)
    x_val = x_val.unsqueeze(0).to(device)
    y_pred_1  = model_LSTM(x_val).unsqueeze(0)

    # add y_pred_1 to the bottom of 'x_val' and remove the first row of original 'x_val'
    x_val_1 = torch.cat((x_val, y_pred_1), dim=1)
    x_val_1 = x_val_1[:,1:,:]
    
    # put x_val_1 into model again and get y_pred_2
    y_pred_2 = model_LSTM(x_val_1).detach().to('cpu').numpy()
    _, y_val_2 = data_val.__getitem__(idx+1)
    
    y_val_2 = y_val_2.detach().to('cpu').numpy()
    
    Y_VAL_2.append(y_val_2)
    Y_PRED_2.append(y_pred_2)

In [13]:
Y_VAL_2 = np.vstack(Y_VAL_2)
Y_PRED_2 = np.vstack(Y_PRED_2)

In [14]:
# calculate the RMSE and MAPE
rmse_flow_2_LSTM = np.sqrt(np.nanmean((Y_VAL_2 - Y_PRED_2) ** 2))
print(rmse_flow_2_LSTM)

mape_flow_2_LSTM = np.nanmean(np.abs((Y_VAL_2 - Y_PRED_2) / Y_VAL_2)) * 100
print(mape_flow_2_LSTM)

8.190036
11.575881


## 9 minutes prediction

In [15]:
# create empty np array
Y_VAL_3 = []
Y_PRED_3 = []

In [16]:
for idx in range(data_val.__len__()-2):
    x_val, _ = data_val.__getitem__(idx)
    x_val = x_val.unsqueeze(0).to(device)
    y_pred_1  = model_LSTM(x_val).unsqueeze(0)

    # add y_pred_1 to the bottom of 'x_val' and remove the first row of original 'x_val'
    x_val_1 = torch.cat((x_val, y_pred_1), dim=1)
    x_val_1 = x_val_1[:,1:,:]
    
    # put x_val_1 into model again and get y_pred_2
    y_pred_2 = model_LSTM(x_val_1).unsqueeze(0)
    
    x_val_2 = torch.cat((x_val_1, y_pred_2), dim=1)
    x_val_2 = x_val_2[:,1:,:]
    
    y_pred_3 = model_LSTM(x_val_2).detach().to('cpu').numpy()
    
    _, y_val_3 = data_val.__getitem__(idx+2)
    
    y_val_3 = y_val_3.detach().to('cpu').numpy()
    
    Y_VAL_3.append(y_val_3)
    Y_PRED_3.append(y_pred_3)

In [17]:
Y_VAL_3 = np.vstack(Y_VAL_3)
Y_PRED_3 = np.vstack(Y_PRED_3)

In [18]:
rmse_flow_3_LSTM = np.sqrt(np.nanmean((Y_VAL_3 - Y_PRED_3) ** 2))
print(rmse_flow_3_LSTM)

mape_flow_3_LSTM = np.nanmean(np.abs((Y_VAL_3 - Y_PRED_3) / Y_VAL_3)) * 100
print(mape_flow_3_LSTM)

7.788554
11.39954


## 12 minutes prediction

In [19]:
# create empty np array
Y_VAL_4 = []
Y_PRED_4 = []

In [20]:
for idx in range(data_val.__len__()-3):
    x_val, _ = data_val.__getitem__(idx)
    x_val = x_val.unsqueeze(0).to(device)
    y_pred_1  = model_LSTM(x_val).unsqueeze(0)

    # add y_pred_1 to the bottom of 'x_val' and remove the first row of original 'x_val'
    x_val_1 = torch.cat((x_val, y_pred_1), dim=1)
    x_val_1 = x_val_1[:,1:,:]
    
    # put x_val_1 into model again and get y_pred_2
    y_pred_2 = model_LSTM(x_val_1).unsqueeze(0)
    
    x_val_2 = torch.cat((x_val_1, y_pred_2), dim=1)
    x_val_2 = x_val_2[:,1:,:]
    
    y_pred_3 = model_LSTM(x_val_2).unsqueeze(0)
    
    x_val_3 = torch.cat((x_val_2, y_pred_3), dim=1)
    x_val_3 = x_val_3[:,1:,:]
    
    y_pred_4 = model_LSTM(x_val_3).detach().to('cpu').numpy()
    
    _, y_val_4 = data_val.__getitem__(idx+3)
    
    y_val_4 = y_val_4.detach().to('cpu').numpy()
    
    Y_VAL_4.append(y_val_4)
    Y_PRED_4.append(y_pred_4)

In [21]:
Y_VAL_4 = np.vstack(Y_VAL_4)
Y_PRED_4 = np.vstack(Y_PRED_4)

In [22]:
rmse_flow_4_LSTM = np.sqrt(np.nanmean((Y_VAL_4 - Y_PRED_4) ** 2))
print(rmse_flow_4_LSTM)

mape_flow_4_LSTM = np.nanmean(np.abs((Y_VAL_4 - Y_PRED_4) / Y_VAL_4)) * 100
print(mape_flow_4_LSTM)

7.4574423
11.25438


## 15 minutes prediction

In [23]:
# create empty np array
Y_VAL_5 = []
Y_PRED_5 = []

In [24]:
for idx in range(data_val.__len__()-4):
    x_val, _ = data_val.__getitem__(idx)
    x_val = x_val.unsqueeze(0).to(device)
    y_pred_1  = model_LSTM(x_val).unsqueeze(0)

    # add y_pred_1 to the bottom of 'x_val' and remove the first row of original 'x_val'
    x_val_1 = torch.cat((x_val, y_pred_1), dim=1)
    x_val_1 = x_val_1[:,1:,:]
    
    # put x_val_1 into model again and get y_pred_2
    y_pred_2 = model_LSTM(x_val_1).unsqueeze(0)
    
    x_val_2 = torch.cat((x_val_1, y_pred_2), dim=1)
    x_val_2 = x_val_2[:,1:,:]
    
    y_pred_3 = model_LSTM(x_val_2).unsqueeze(0)
    
    x_val_3 = torch.cat((x_val_2, y_pred_3), dim=1)
    x_val_3 = x_val_3[:,1:,:]
    
    y_pred_4 = model_LSTM(x_val_3).unsqueeze(0)
    
    x_val_4 = torch.cat((x_val_3, y_pred_4), dim=1)
    x_val_4 = x_val_4[:,1:,:]
    
    y_pred_5 = model_LSTM(x_val_4).detach().to('cpu').numpy()
    
    _, y_val_5 = data_val.__getitem__(idx+4)
    
    y_val_5 = y_val_5.detach().to('cpu').numpy()
    
    Y_VAL_5.append(y_val_5)
    Y_PRED_5.append(y_pred_5)

In [25]:
Y_VAL_5 = np.vstack(Y_VAL_5)
Y_PRED_5 = np.vstack(Y_PRED_5)

In [26]:
rmse_flow_5_LSTM = np.sqrt(np.nanmean((Y_VAL_5 - Y_PRED_5) ** 2))
print(rmse_flow_5_LSTM)

mape_flow_5_LSTM = np.nanmean(np.abs((Y_VAL_5 - Y_PRED_5) / Y_VAL_5)) * 100
print(mape_flow_5_LSTM)

7.2076445
11.143913


In [33]:
import pandas as pd

In [34]:
data = {
    'Prediction Length': ['RMSE(%)', 'MAPE(%)'],
    '3-min prediction': [rmse_flow_1_LSTM, mape_flow_1_LSTM],
    '6-min prediction': [rmse_flow_2_LSTM, mape_flow_2_LSTM],
    '9-min prediction': [rmse_flow_3_LSTM, mape_flow_3_LSTM],
    '12-min prediction': [rmse_flow_4_LSTM, mape_flow_4_LSTM],
    '15-min prediction': [rmse_flow_5_LSTM, mape_flow_5_LSTM]
}
df = pd.DataFrame(data)
df.iloc[:, 1:] = df.iloc[:, 1:].round(2)
df.to_csv('Tables/Prediction_Error_LSTM.csv', index=False)